In [1]:
from datasets import load_dataset, concatenate_datasets

In [2]:
# Flickr30k датасет - состоит из 30 000 изображений, каждое из которых связано с пятью разными подписями, 
# обеспечивающими точное описание основных объектов и событий. В них нет известных людей или мест. 
# Демонстрирует разных сцен и ситуаций.
ds_flickr30k = load_dataset("lmms-lab/flickr30k")

ds_flickr30k

DatasetDict({
    test: Dataset({
        features: ['image', 'caption', 'sentids', 'img_id', 'filename'],
        num_rows: 31783
    })
})

In [3]:
# Функция поиска в подписях изображений упоминаний про детей, младенцев, мальчиков, девочек
def check_for_children(caption_list):
    keywords = ["child", "girl", "boy", "baby"]
    return any(
        keyword in caption.lower() for keyword in keywords for caption in caption_list
    )

In [4]:
# Бинарная разметка датасета по наличию детей
ds_flickr30k["test"] = ds_flickr30k["test"].add_column(
    "has_children",
    [check_for_children(example["caption"]) for example in ds_flickr30k["test"]],
)

# Фильтрация датасета только на детей
filter_ds_flickr30k = ds_flickr30k["test"].filter(
    lambda example: example["has_children"]
)

# Удаление лишней информации
filter_ds_flickr30k = filter_ds_flickr30k.remove_columns(
    ["sentids", "has_children", "img_id", "filename"]
)

filter_ds_flickr30k

Dataset({
    features: ['image', 'caption'],
    num_rows: 10361
})

In [5]:
# NoCaps датасет - состоит из 166 100 созданных человеком подписей, описывающих 15 100 изображений 
# из проверочных и тестовых наборов Open Images.
ds_nocaps = load_dataset("lmms-lab/NoCaps")

ds_nocaps

DatasetDict({
    validation: Dataset({
        features: ['image', 'image_coco_url', 'image_date_captured', 'image_file_name', 'image_height', 'image_width', 'image_id', 'image_license', 'image_open_images_id', 'annotations_ids', 'annotations_captions'],
        num_rows: 4500
    })
    test: Dataset({
        features: ['image', 'image_coco_url', 'image_date_captured', 'image_file_name', 'image_height', 'image_width', 'image_id', 'image_license', 'image_open_images_id', 'annotations_ids', 'annotations_captions'],
        num_rows: 10600
    })
})

In [6]:
# Бинарная разметка датасета по наличию детей
ds_nocaps["validation"] = ds_nocaps["validation"].add_column(
    "has_children",
    [
        check_for_children(example["annotations_captions"])
        for example in ds_nocaps["validation"]
    ],
)

# Фильтрация датасета только на детей
filter_ds_nocaps = ds_nocaps["validation"].filter(
    lambda example: example["has_children"]
)

# Удаление лишней информации
filter_ds_nocaps = filter_ds_nocaps.remove_columns(
    [
        "annotations_ids",
        "has_children",
        "image_open_images_id",
        "image_license",
        "image_width",
        "image_height",
        "image_date_captured",
        "image_coco_url",
        "image_file_name",
        "image_id",
    ]
)

# Переименование колонок под соответствие первому датасету
filter_ds_nocaps = filter_ds_nocaps.rename_columns({"annotations_captions": "caption"})

filter_ds_nocaps

Dataset({
    features: ['image', 'caption'],
    num_rows: 667
})

In [7]:
# Объединение дву датасетов для дообучения моделей
ds_train = concatenate_datasets([filter_ds_flickr30k, filter_ds_nocaps])

ds_train

Dataset({
    features: ['image', 'caption'],
    num_rows: 11028
})

In [8]:
ds_train.save_to_disk("datasets/ds_train")

Saving the dataset (0/4 shards):   0%|          | 0/11028 [00:00<?, ? examples/s]